In [1]:
import pandas as pd
import numpy as np
import sqlite3
conn = sqlite3.connect('consulates.sqlite')

In [2]:
df = pd.read_pickle("all_months.pkl")
DATA_START_DATE = df["Month"].min()
DATA_END_DATE = df["Month"].max()
COVID_START_DATE = pd.Timestamp("20200301")
NORMAL_MONTHS = pd.date_range(start=DATA_START_DATE, end=COVID_START_DATE, freq="MS", name="Month", closed="left")
COVID_MONTHS = pd.date_range(start=COVID_START_DATE, end=DATA_END_DATE, freq="MS", name="Month")
NORMAL_MONTHS, COVID_MONTHS

(DatetimeIndex(['2017-03-01', '2017-04-01', '2017-05-01', '2017-06-01',
                '2017-07-01', '2017-08-01', '2017-09-01', '2017-10-01',
                '2017-11-01', '2017-12-01', '2018-01-01', '2018-02-01',
                '2018-03-01', '2018-04-01', '2018-05-01', '2018-06-01',
                '2018-07-01', '2018-08-01', '2018-09-01', '2018-10-01',
                '2018-11-01', '2018-12-01', '2019-01-01', '2019-02-01',
                '2019-03-01', '2019-04-01', '2019-05-01', '2019-06-01',
                '2019-07-01', '2019-08-01', '2019-09-01', '2019-10-01',
                '2019-11-01', '2019-12-01', '2020-01-01', '2020-02-01'],
               dtype='datetime64[ns]', name='Month', freq='MS'),
 DatetimeIndex(['2020-03-01', '2020-04-01', '2020-05-01', '2020-06-01',
                '2020-07-01', '2020-08-01', '2020-09-01', '2020-10-01',
                '2020-11-01', '2020-12-01', '2021-01-01', '2021-02-01',
                '2021-03-01', '2021-04-01'],
               dtype='dat

In [3]:
df["Visa Class"] = df["Visa Class"].replace(to_replace=["CR1", "IR1"], value="CR1/IR1")
df["Visa Class"] = df["Visa Class"].replace(to_replace=["CR2", "IR2"], value="CR2/IR2")
df["Post"] = df["Post"].replace(to_replace={"AIT Taipei": "Taipei", "Dar Es Salaam": "Dar es Salaam", "Guatemala": "Guatemala City", "Panama": "Panama City", "Kuwait": "Kuwait City", "Port Of Spain": "Port of Spain", "Rio De Janeiro": "Rio de Janeiro"})
df = df.groupby(["Post", "Visa Class", "Month"], as_index=False, sort=False)["Issuances"].sum()
df

,Post,Visa Class,Month,Issuances
0,Dublin,H4,2017-03-01,1
1,Warsaw,K1,2017-03-01,10
2,Warsaw,J2,2017-03-01,6
3,Warsaw,J1,2017-03-01,477
4,Warsaw,I,2017-03-01,15
...,...,...,...,...
272039,Bogota,I5,2021-04-01,4
272040,Bogota,FX,2021-04-01,22
272041,Bogota,F4,2021-04-01,9
272042,Bogota,F3,2021-04-01,13


In [4]:
import re

def slugify(text: str) -> str:
    text = text.strip().lower()
    text = re.sub(r"[\s_]", "-", text)
    text = re.sub(r"[^\w-]", "", text)
    return text

df["Post Slug"] = df["Post"].apply(slugify)
df["Visa Class Slug"] = df["Visa Class"].apply(slugify)

post_slugs = pd.DataFrame(df[["Post Slug", "Post"]].drop_duplicates().set_index("Post Slug"))
post_slugs.to_sql("post_slugs", conn, if_exists="replace")

visa_slugs = pd.DataFrame(df[["Visa Class Slug", "Visa Class"]].drop_duplicates().set_index("Visa Class Slug"))
visa_slugs.to_sql("visa_slugs", conn, if_exists="replace")

/Users/underyx/.local/pipx/venvs/jupyter/lib/python3.9/site-packages/pandas/core/generic.py:2779: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


In [5]:
def get_baseline(df: pd.DataFrame) -> pd.DataFrame:
    df = df.set_index("Month")
    return df[["Issuances"]].reindex(index=NORMAL_MONTHS, fill_value=0).mean()

baselines = df.groupby(["Post Slug", "Visa Class Slug"], sort=False).apply(get_baseline)
baselines

Issuances
Post Slug  Visa Class Slug            
dublin     h4                 8.583333
warsaw     k1                16.111111
           j2                13.666667
           j1               232.722222
           i                 11.833333
...                                ...
bridgetown f4                 0.000000
           f2b                0.000000
bogota     f2a                0.000000
bratislava fx                 0.000000
bogota     i5                 0.000000

[15314 rows x 1 columns]

In [6]:
df.loc[df["Post Slug"] == "guayaquil"].loc[df["Visa Class Slug"] == "f3"]

,Post,Visa Class,Month,Issuances,Post Slug,Visa Class Slug
270344,Guayaquil,F3,2021-04-01,9,guayaquil,f3


In [7]:
baselines.loc[("budapest", "cr1ir1")]

Issuances    3.361111
Name: (budapest, cr1ir1), dtype: float64

In [8]:
baselines.to_pickle("baselines.pkl")
baselines.to_sql("baselines", conn, if_exists="replace")

/Users/underyx/.local/pipx/venvs/jupyter/lib/python3.9/site-packages/pandas/core/generic.py:2779: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


In [9]:
def get_backlog(df: pd.DataFrame) -> pd.DataFrame:
    df = df.reset_index()
    key = tuple(df.loc[0, ["Post Slug", "Visa Class Slug"]])
    baseline = baselines.loc[key][0]

    df = df.set_index("Month")

    normal_months = df[["Issuances"]].reindex(index=NORMAL_MONTHS, fill_value=0)

    covid_months = df[["Issuances"]].reindex(index=COVID_MONTHS, fill_value=0)
    covid_months["Actual Progress"] = covid_months["Issuances"].cumsum()
    covid_months["Expected Progress"] = np.linspace(baseline, baseline * len(covid_months), num=len(covid_months))
    covid_months["Backlog"] = covid_months["Actual Progress"] - covid_months["Expected Progress"]
    covid_months["Months Ahead"] = covid_months["Backlog"] / baseline
    covid_months = covid_months.drop(columns=["Actual Progress", "Expected Progress"])

    return pd.concat([normal_months, covid_months])[["Issuances", "Backlog", "Months Ahead"]]

backlogs = df.groupby(["Post Slug", "Visa Class Slug"], sort=False).apply(get_backlog)

In [ ]:
backlogs.to_pickle("backlogs.pkl")

In [ ]:
backlogs.to_sql("backlogs", conn, if_exists="replace")